In [1]:
import random
import numpy as np
import tabulate
tabulate.PRESERVE_WHITESPACE = True

np.set_printoptions(precision=5, suppress=True)  # set print options
precision = 0.00001

P = np.array([
    [0.787, 0,      0.213],
    [0.263, 0.174,  0.563],
    [0.192, 0,      0.808]
])

def generate_state_matrices(initial_matrix, deviation=0.00001):
    prev_matrix = initial_matrix.copy()
    while True:
        current_matrix = prev_matrix @ initial_matrix
        step_diff = abs(current_matrix - prev_matrix).max()
        prev_matrix = current_matrix
        yield current_matrix, step_diff
        if step_diff <= deviation:
            break

In [2]:
# Задание 1. Матрицы переходных вероятностей за n шагов
results = [[1, P, '-']]
for state_matrix, state_matrix_diff in generate_state_matrices(P):
    results.append([
        results[-1][0] + 1,
        state_matrix,
        f'{state_matrix_diff:10.6f}'
    ])
print(tabulate.tabulate(results, tablefmt='grid', headers=['n', 'P^n', 'delta']))

+-----+-----------------------------+------------+
|   n | P^n                         | delta      |
+=====+=============================+============+
|   1 | [[0.787 0.    0.213]        | -          |
|     |  [0.263 0.174 0.563]        |            |
|     |  [0.192 0.    0.808]]       |            |
+-----+-----------------------------+------------+
|   2 | [[0.66027 0.      0.33974]  |   0.143724 |
|     |  [0.36084 0.03028 0.60889]  |            |
|     |  [0.30624 0.      0.69376]] |            |
+-----+-----------------------------+------------+
|   3 | [[0.58486 0.      0.41514]  |   0.075407 |
|     |  [0.40885 0.00527 0.58588]  |            |
|     |  [0.37421 0.      0.62579]] |            |
+-----+-----------------------------+------------+
|   4 | [[0.53999 0.      0.46001]  |   0.044867 |
|     |  [0.43564 0.00092 0.56344]  |            |
|     |  [0.41466 0.      0.58534]] |            |
+-----+-----------------------------+------------+
|   5 | [[0.51329 0.      0.486

In [3]:
# Задание 2. Стационарное распределение вероятностей
I = np.eye(max(P.shape))
right = np.array([0,0,1])
eq = P.T - I

extended_matrix = np.vstack((eq, np.array([1,1,1])))
stationary_distribution = np.linalg.solve(extended_matrix[1:4], right)
print(f'Стационарное распределение: {stationary_distribution}')

Стационарное распределение: [ 0.47407 -0.       0.52593]


In [4]:
# Проверка стационарности найденного распределения
print(f'stationary_distribution @ P: {stationary_distribution @ P}')

stationary_distribution @ P: [0.47407 0.      0.52593]


In [5]:
# Задание 3 Распределения вероятностей состояний через n шагов
initial_distributions = [
    np.array([1, 0, 0]),
    np.array([0, 1, 0]),
    np.array([0, 0, 1])
]
for initial_distribution in initial_distributions:
    i = 1
    table_data = []
    current_distribution = initial_distribution
    while True:
        delta = (current_distribution - stationary_distribution).max()
        if delta <= precision:
            break
        table_data.append([
            i,
            current_distribution,
            f'd={delta:10.5f} '
        ])
        current_distribution = current_distribution @ P
        i += 1

    print(f'# P_i = {initial_distribution}')
    print(tabulate.tabulate(table_data, tablefmt='grid', headers=['n', '(p_1(n), p_2(n), p_3(n))', 'delta_n']))

# P_i = [1 0 0]
+-----+----------------------------+---------------+
|   n | (p_1(n), p_2(n), p_3(n))   | delta_n       |
+=====+============================+===============+
|   1 | [1 0 0]                    | d=   0.52593  |
+-----+----------------------------+---------------+
|   2 | [0.787 0.    0.213]        | d=   0.31293  |
+-----+----------------------------+---------------+
|   3 | [0.66027 0.      0.33974]  | d=   0.18619  |
+-----+----------------------------+---------------+
|   4 | [0.58486 0.      0.41514]  | d=   0.11078  |
+-----+----------------------------+---------------+
|   5 | [0.53999 0.      0.46001]  | d=   0.06592  |
+-----+----------------------------+---------------+
|   6 | [0.51329 0.      0.48671]  | d=   0.03922  |
+-----+----------------------------+---------------+
|   7 | [0.49741 0.      0.50259]  | d=   0.02334  |
+-----+----------------------------+---------------+
|   8 | [0.48796 0.      0.51204]  | d=   0.01388  |
+-----+-----------------------

In [6]:
# Задание 4
part4_results = {}
for initial_position in [1, 2, 3]:
    p4_result = []
    R = 0
    n = 0
    current_position = initial_position
    while True:
        rand = random.random()
        if rand < P[current_position - 1][0]:
            current_position = 1
        elif rand < P[current_position - 1][0] + P[current_position - 1][1]:
            current_position = 2
        else:
            current_position = 3
        n += 1
        if current_position == initial_position:
            R += 1
        v = R/n
        delta = abs(v - stationary_distribution[initial_position-1])
        p4_result.append([n, R, v, delta])
        if delta <= 0.001:
            break

    part4_results[initial_position] = p4_result
    print(f'N_min({initial_position})={p4_result[-1][0]}')

N_min(1)=688805
N_min(2)=1
N_min(3)=1452


In [7]:
# Задание 5
for initial_position in [1, 2, 3]:
    print(f'# Initial position = {initial_position}')
    print(tabulate.tabulate(part4_results[initial_position][:16], tablefmt='grid', headers=['n', 'R(i,n)', 'v(i, n)', 'delta']))

# Initial position = 1
+-----+----------+-----------+-----------+
|   n |   R(i,n) |   v(i, n) |     delta |
+=====+==========+===========+===========+
|   1 |        1 |  1        | 0.525926  |
+-----+----------+-----------+-----------+
|   2 |        2 |  1        | 0.525926  |
+-----+----------+-----------+-----------+
|   3 |        3 |  1        | 0.525926  |
+-----+----------+-----------+-----------+
|   4 |        3 |  0.75     | 0.275926  |
+-----+----------+-----------+-----------+
|   5 |        3 |  0.6      | 0.125926  |
+-----+----------+-----------+-----------+
|   6 |        3 |  0.5      | 0.0259259 |
+-----+----------+-----------+-----------+
|   7 |        3 |  0.428571 | 0.0455026 |
+-----+----------+-----------+-----------+
|   8 |        3 |  0.375    | 0.0990741 |
+-----+----------+-----------+-----------+
|   9 |        3 |  0.333333 | 0.140741  |
+-----+----------+-----------+-----------+
|  10 |        3 |  0.3      | 0.174074  |
+-----+----------+-----------+-